In [ ]:
!pip install transformers datasets peft torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
import torch

In [ ]:
dataset = load_dataset("huggingartists/kendrick-lamar")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.22k [00:00<?, ?B/s]

kendrick-lamar.py:   0%|          | 0.00/4.08k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/1.42M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/861 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 861
    })
})

In [ ]:
dataset["train"][0]

{'text': 'Nobody pray for me\nIt been that day for me\nWay \nAyy, I remember syrup sandwiches and crime allowances\nFinesse a nigga with some counterfeits, but now Im countin’ this\nParmesan where my accountant lives, in fact Im downin this\nD’USSÉ with my boo bae tastes like Kool-Aid for the analysts\nGirl, I can buy your ass the world with my paystub\nOoh, that pussy good, wont you sit it on my taste bloods?\nI get way too petty once you let me do the extras\nPull up on your block, then break it down: we playin Tetris\nAM to the PM, PM to the AM, funk\nPiss out your per diem, you just gotta hate em, funk\nIf I quit your BM, I still ride Mercedes, funk\nIf I quit this season, I still be the greatest, funk\nMy left stroke just went viral\nRight stroke put lil baby in a spiral\nSoprano C, we like to keep it on a high note\nIts levels to it, you and I know\nBitch, be humble \nSit down \nBe humble \nSit down \nBe humble \nBitch, sit down \nBe humble \nSit down \nBe humble \nSit down \nBe 

In [ ]:
dataset["train"][1]

{'text': 'I love bad bitches, thats my fuckin problem \nAnd yeah, I like to fuck, I got a fuckin problem \nI love bad bitches, thats my fuckin problem \nAnd yeah, I like to fuck, I got a fuckin problem \nI love bad bitches, thats my fuckin problem \nAnd yeah, I like to fuck, I got a fuckin problem \nIf findin somebody real is your fuckin problem \nBring your girls to the crib, maybe we can solve it, ayy\nHold up, bitches simmer down \nTakin hella long, bitch, give it to me now \nMake that thing pop like a semi or a 9\nOoh, baby like it raw with the shimmy shimmy ya, huh?\nA$AP , get like me \nNever met a motherfucker fresh like me \nAll these motherfuckers wanna dress like me \nBut the chrome to your dome make you sweat like Keith\n‘Cause Im the nigga, the nigga, nigga, like how you figure? \nGettin figures and fuckin bitches, she rollin Swishers\nBrought her bitches, I brought my niggas \nThey gettin bent up off the liquor \nShe love my licorice, I let her lick it \nThey say money mak

In [ ]:
tokeniser = AutoTokenizer.from_pretrained("gpt2", use_fast=True)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
tokeniser.pad_token = tokeniser.eos_token

In [ ]:
def tokenise_function(examples):
  return tokeniser(examples['text'])

In [ ]:
tokenised_dataset = dataset.map(tokenise_function, batched=True, num_proc=4, remove_columns=['text'])

Map (num_proc=4):   0%|          | 0/861 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1053 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1061 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1202 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1044 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
tokenised_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 861
    })
})

In [ ]:
tokenised_dataset['train']['input_ids'][0]

[24795,
 12472,
 329,
 502,
 198,
 1026,
 587,
 326,
 1110,
 329,
 502,
 198,
 25309,
 220,
 198,
 32,
 22556,
 11,
 314,
 3505,
 24208,
 29777,
 290,
 4065,
 42290,
 198,
 37,
 1127,
 325,
 257,
 44873,
 4908,
 351,
 617,
 29465,
 896,
 11,
 475,
 783,
 1846,
 954,
 259,
 447,
 247,
 428,
 198,
 47,
 1670,
 42890,
 810,
 616,
 42627,
 3160,
 11,
 287,
 1109,
 1846,
 866,
 259,
 428,
 198,
 35,
 447,
 247,
 32835,
 38351,
 351,
 616,
 22513,
 275,
 3609,
 18221,
 588,
 509,
 970,
 12,
 44245,
 329,
 262,
 13440,
 198,
 24151,
 11,
 314,
 460,
 2822,
 534,
 840,
 262,
 995,
 351,
 616,
 1414,
 301,
 549,
 198,
 46,
 1219,
 11,
 326,
 18185,
 922,
 11,
 28329,
 345,
 1650,
 340,
 319,
 616,
 6938,
 2910,
 82,
 30,
 198,
 40,
 651,
 835,
 1165,
 25229,
 1752,
 345,
 1309,
 502,
 466,
 262,
 33849,
 198,
 42940,
 510,
 319,
 534,
 2512,
 11,
 788,
 2270,
 340,
 866,
 25,
 356,
 711,
 259,
 27351,
 2442,
 198,
 2390,
 284,
 262,
 3122,
 11,
 3122,
 284,
 262,
 3001,
 11,
 46212,
 198,
 47,


In [ ]:
block_size = 128

In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # assuming each key have list (their values) of same length, get the length of the lists
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # Floor divide to get perfect divisor of block_size and multiply back to get total length (drop remainder)
    # We could add padding if the model supported it instead of dropping.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {}
    for k,t in concatenated_examples.items():
        chunks = [t[i: i + block_size] for i in range(0, total_length, block_size)]

        # check if last chunk needs padding
        if len(chunks[-1]) < block_size:
            num_padding = block_size - len(chunks[-1])
            # denote padding_value = 0, and add padding to make the chunk size = block_size
            chunks[-1] = chunks[-1] + [0] * num_padding

        result[k] = chunks
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenised_dataset.map(
    group_texts,
    batched=True,
    batch_size=64,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/861 [00:00<?, ? examples/s]

In [ ]:
lm_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5163
    })
})

In [ ]:
tokeniser.decode(lm_datasets["train"][1]["input_ids"])


' up on your block, then break it down: we playin Tetris\nAM to the PM, PM to the AM, funk\nPiss out your per diem, you just gotta hate em, funk\nIf I quit your BM, I still ride Mercedes, funk\nIf I quit this season, I still be the greatest, funk\nMy left stroke just went viral\nRight stroke put lil baby in a spiral\nSoprano C, we like to keep it on a high note\nIts levels to it, you and I know\nBitch, be humble \nSit down \nBe humble \nSit'

In [ ]:
model = AutoModelForCausalLM.from_pretrained("gpt2")

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
training_args = TrainingArguments(
    f"gpt2-finetuned-kendrick_lamar",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=3e-4,
    weight_decay=1e-4,
    report_to="none"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
)

In [ ]:
trainer.train()


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
500,3.617200
1000,2.866700
1500,2.377200


TrainOutput(global_step=1610, training_loss=2.90270082461908, metrics={'train_runtime': 1164.2069, 'train_samples_per_second': 22.174, 'train_steps_per_second': 1.383, 'total_flos': 1681414225920000.0, 'train_loss': 2.90270082461908, 'epoch': 4.985282726568552})

In [ ]:
model.save_pretrained("gpt2-finetuned-kendrick_lamar/final_model")

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
model2 = AutoModelForCausalLM.from_pretrained("gpt2-finetuned-kendrick_lamar/final_model", device_map="auto")

In [ ]:
torch.manual_seed(42)


In [ ]:
input_ids = tokeniser.encode("Write me a song \n", return_tensors="pt").to(device)


In [ ]:
input_ids

tensor([[16594,   502,   257,  3496,   220,   198]], device='cuda:0')

In [ ]:
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=500,
    top_k=50,
    top_p=0.95,
    temperature= 0.9,
    repetition_penalty = 1.1
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
sample_output.shape

torch.Size([1, 500])

In [ ]:
generated_text = tokeniser.decode(sample_output[0], skip_special_tokens=True)
print(generated_text)

Write me a song 
Baby, I know you wanna be mine 
Baby...here we go again
I need all of mines
Please come here quick, please bring the coffin
Please bring the morgue with it
We dont wanna see you dead, we wont lie
God is fair and you will find him
In the heart of an un-American
How can I help you if my ethnicity isnt strong?
I mean I can help you even if my race is not strong?
If I hate you, Ill make sure your love is sweet as Sounwave
I am an African American, you are African
And I feel that every message you send is wrapped in plastic
With your hands out the window, you might catch an audible hummer
Throw the plastic back at me, like Titanic to a Haitian
My cousins relatives die before they turn us, I wonder how Im posed to respond
We was raised by wolves, were old enough
Not prepared, not to feed the hoes, not willing to wait for nothing
So there was nothing to take life
But the anticipation, wait what the hell, our familys happening over a year, we panic over nothing
I wonder when t

In [ ]:
with open("gpt2_text_output.txt", "w") as f:
    f.write(generated_text)